In [ ]:
# /**********************************************************************************************************

# File Name		: 	image_load.ipynb
# Purpose			:   Loading image data with simulated attributes into bigquery table
# Author			:   DeepSphere.AI, Inc.
# Date and Time 	: 	03/16/2021 10:30 hrs
# Version			: 	1.0 

# /************************************************************************************************************



def image_load():
    from google.cloud import storage
    import numpy as np
    import pandas as pd
    import datetime
    import time
    vAR_client = storage.Client()
    vAR_columns = ['text_content','age','customer_id','gender']
    vAR_text_list = []
    vAR_cust_id = ['John','Mike','Lisa','Peter','Michael','Anderson','Virat','Archer','Gilchrist','Ponting']
    vAR_age = [12,35,23,54,34,29,27,67,15,26]
    vAR_gender = ['Male','Male','Female','Male','Male','Male','Male','Male','Male','Male']
    vAR_bname = 'videoai_intelligence'
    for lCount_blob in vAR_client.list_blobs(vAR_bname,prefix='VideoAI/covid_image'):
        vAR_fname = 'gs://'+vAR_bname+'/'+lCount_blob.name
        vAR_text = detect_text_uri(vAR_fname)
        vAR_text_list.append(vAR_text)
    vAR_df= pd.DataFrame(columns = vAR_columns)
    vAR_df['text_content'] = vAR_text_list
    vAR_df['age'] = vAR_age
    vAR_df['age'] = vAR_df.age.astype(str)
    vAR_df['customer_id'] = vAR_cust_id
    vAR_df['gender'] = vAR_gender
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
    vAR_df['created_time'] = st
    vAR_df['language'] = 'en'
    vAR_df['platform_source'] = 'facebook'
    vAR_df['text_content'] = vAR_df.text_content.astype(str)
    vAR_image_flag = bq_insert(vAR_df)
    return vAR_image_flag

In [ ]:
def detect_text_uri(uri):
    """Detects text in the file located in Google Cloud Storage or on the Web.
    """
    from google.cloud import vision
    import json
    vAR_client = vision.ImageAnnotatorClient()
    vAR_image = vision.Image()
    vAR_image.source.image_uri = uri

    vAR_response = vAR_client.text_detection(image=vAR_image)
    vAR_texts = vAR_response.text_annotations
    vAR_text_content = []
    boolean = False
    for lCount_text in vAR_texts:
        if not boolean:
            boolean = True
            vAR_txt = str(lCount_text.description)
            vAR_txt = vAR_txt.replace('[','').replace(']','')
            vAR_text_content.append(vAR_txt.replace('\n',' ')) 
    if vAR_response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                vAR_response.error.message))
    return vAR_text_content

In [ ]:
def bq_insert(df):
    from google.cloud import bigquery
    # Construct a BigQuery client object.
    vAR_bq_client = bigquery.Client()
    # TODO(developer): Set table_id to the ID of the table to create.
    vAR_table_id = "vast-verve-292018.ds_ai.ds_ai_image_data"
    vAR_job_config = bigquery.LoadJobConfig(
        vAR_schema = [bigquery.SchemaField("text_content", bigquery.enums.SqlTypeNames.STRING)],
         write_disposition="WRITE_APPEND")
    vAR_job = vAR_bq_client.load_table_from_dataframe(
        df, vAR_table_id, job_config=vAR_job_config
    )  # Make an API request.
    vAR_job.result()  # Wait for the job to complete.

    vAR_table = vAR_bq_client.get_table(vAR_table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            vAR_table.num_rows, len(vAR_table.schema), vAR_table_id
        )
    )
    return True

In [ ]:
# /**********************************************************************************************************
# Disclaimer.

# We are providing this code block strictly for learning and researching, this is not a production ready code.
# We have no liability on this particular code under any circumstances; users should
# use this code on their own risk. All software,hardware and other products that are
# referenced in these materials belong to the respective vendor who developed or who owns 
# this product.
# /**********************************************************************************************************
